In [1]:
## 성능개선
## base line 모델 만들기
import pandas as pd
import numpy as np
import warnings


warnings.filterwarnings(action='ignore')

data_path= './'

sales_train = pd.read_csv(data_path + 'sales_train.csv')
shops = pd.read_csv(data_path + 'shops.csv')
items = pd.read_csv(data_path +'items.csv')
item_categories = pd.read_csv(data_path + 'item_categories.csv')
test= pd.read_csv(data_path + 'test.csv')
submission= pd.read_csv(data_path + 'sample_submission.csv')

In [2]:
## 피처 엔지니어링 : 피처명 한글화
sales_train = sales_train.rename(
    columns = {
        'date' : '날짜',
        'date_block_num' : '월id',
        'shop_id': '상점id',
        'item_id' : '상품id',
        'item_price' : '판매가',
        'item_cnt_day' : '판매량'
    }
)

## shops 변경
shops=shops.rename(
    columns={
            'shop_name' : '상점명',
            'shop_id': '상점id'
        })

# items 변경
items=items.rename(
    columns={
            'item_name' : '상품명',
            'item_id': '상품id',
            'item_category_id':'상품분류id'
        })

#item_categories 변경
item_categories=item_categories.rename(
    columns={
            'item_category_name' : '상품분류명',
            'item_category_id': '상품분류id',
        })

## tets 피처명 변경
test=test.rename(
    columns={
        'shop_id' : '상점id',
        'item_id' : '상품id'
    }
)

In [3]:
def downcast(df,verbose=True):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast = 'integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast = 'float')
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print(f'{100 * (start_mem - end_mem)/start_mem : .1f}% 압축됨')
    
    return df


all_df = [sales_train , shops , items , item_categories, test]
for df in all_df:
    df  = downcast(df)

 62.5% 압축됨
 38.6% 압축됨
 54.2% 압축됨
 39.9% 압축됨
 70.8% 압축됨


In [4]:
## sales _train 이상치 제거 및 전처리
## 판매가와 판매량의 이상치 제거

# 판매가가 0보다 크고 50000보다 작은 데이터 추출
sales_train = sales_train.loc[(sales_train['판매가']>0) & (sales_train['판매가']<50000)]

# 판매량이 0보다 크고 1000보다 작은 데이터 추출
sales_train = sales_train.loc[(sales_train['판매량']>0) & (sales_train['판매량']< 1000)]

sales_train

,날짜,월id,상점id,상품id,판매가,판매량
0,02.01.2013,0,59,22154,999.000000,1
1,03.01.2013,0,25,2552,899.000000,1
3,06.01.2013,0,25,2554,1709.050049,1
4,15.01.2013,0,25,2555,1099.000000,1
5,10.01.2013,0,25,2564,349.000000,1
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.000000,1
2935845,09.10.2015,33,25,7460,299.000000,1
2935846,14.10.2015,33,25,7459,349.000000,1
2935847,22.10.2015,33,25,7440,299.000000,1


In [5]:
##상정명을 가지고 전처리
# 같은 상점명인데 다르게 표시되어 있는 것을  합침

print(shops.loc[0,'상점명'] , "||" , shops.loc[57,'상점명'])
print(shops.loc[1,'상점명'] , "||" , shops.loc[58,'상점명'])
print(shops.loc[10,'상점명'] , "||" , shops.loc[11,'상점명'])
print(shops.loc[39,'상점명'] , "||" , shops.loc[40,'상점명'])

##  sales_train 에서 상점 ID 수정
sales_train.loc[sales_train['상점id']==0,'상점id']=57
sales_train.loc[sales_train['상점id']==1,'상점id']=58
sales_train.loc[sales_train['상점id']==10,'상점id']=11
sales_train.loc[sales_train['상점id']==39,'상점id']=40

# test 데이터에서 상점 id 수정
test.loc[test['상점id']== 0 , '상점id'] = 57
test.loc[test['상점id']== 1 , '상점id'] = 58
test.loc[test['상점id']== 10 , '상점id'] = 11
test.loc[test['상점id']== 39 , '상점id'] = 40

## shops 파생 피처 생성 및 인코딩
## 상점명의 첫 단어는 상점이 위치는 도시를 뜻함

shops['도시']=shops['상점명'].apply(lambda x:x.split()[0])
shops['도시'].unique()


# 느낌표가 있는 도시 전처리
shops.loc[shops['도시'] == '!Якутск','도시']='Якутск'
shops['도시'].unique()


## 도시명은 범주형 피처 이므로 문자를 인식하지 못함
## 레이블 인코딩
from sklearn.preprocessing import LabelEncoder

#레이블 인코더 생성
label_encoder = LabelEncoder()

#도시 피처 레이블 인코딩
shops['도시']=label_encoder.fit_transform(shops['도시'])

# 필요없는 상점명 피처제거
shops = shops.drop(columns=['상점명'] , axis=1)
shops.head()


sales_train.groupby('상품id',as_index=False).agg(
    첫판매월 = ('월id','min')
)

!Якутск Орджоникидзе, 56 фран || Якутск Орджоникидзе, 56
!Якутск ТЦ "Центральный" фран || Якутск ТЦ "Центральный"
Жуковский ул. Чкалова 39м? || Жуковский ул. Чкалова 39м²
РостовНаДону ТРК "Мегацентр Горизонт" || РостовНаДону ТРК "Мегацентр Горизонт" Островной


,상품id,첫판매월
0,0,20
1,1,15
2,2,19
3,3,18
4,4,20
...,...,...
21797,22165,7
21798,22166,9
21799,22167,8
21800,22168,0


In [6]:
## items 파생 피처 생성
# 첫 판매월 피처 생성

# 상품명 제거
# items = items.drop(columns = ['상품명'] , axis=1)

# 상품이 판매되었던 첫 달을 구함
# sales_train으로 부터 계산함
items['첫판매월']=sales_train.groupby('상품id',as_index=False).agg(
    첫판매월 = ('월id','min')
)['첫판매월']

# 결측값 확인함
items.isna().sum()


## 테스트 데이터로 예측하는 시점이 첫 판매일이라고 생각하여 데이터 대체함
items.loc[items['첫판매월'].isna()]

## 34로 대체함
items['첫판매월']=items['첫판매월'].fillna(34)
items.isna().sum()

상품명       0
상품id      0
상품분류id    0
첫판매월      0
dtype: int64

In [7]:
## item_categories 파생 피처 생성 및 인코딩
# 상품 분류명의 첫 단어가 범주 대분류 라는 것을 발견함

item_categories['대분류']=item_categories['상품분류명'].apply(lambda x:x.split()[0])
item_categories['대분류'].value_counts()

## 대분류의 갯수가 5개 미만은 etc로 처리함
def make_etc(x):
    if len(item_categories.loc[item_categories['대분류']==x])>=5:
        return x
    else:
        return 'etc'

item_categories['대분류']=item_categories['대분류'].apply(make_etc)
item_categories

,상품분류명,상품분류id,대분류
0,PC - Гарнитуры/Наушники,0,etc
1,Аксессуары - PS2,1,Аксессуары
2,Аксессуары - PS3,2,Аксессуары
3,Аксессуары - PS4,3,Аксессуары
4,Аксессуары - PSP,4,Аксессуары
...,...,...,...
79,Служебные,79,etc
80,Служебные - Билеты,80,etc
81,Чистые носители (шпиль),81,etc
82,Чистые носители (штучные),82,etc


In [8]:
## label encoding
from sklearn.preprocessing import LabelEncoder

# 레이블 인코더 생성
label_encdoer = LabelEncoder()

#대분류 피처 레이블 인코딩
item_categories['대분류']= label_encoder.fit_transform(item_categories['대분류'])
item_categories['대분류'].unique()

# 상품분류명 피처 제거
item_categories=item_categories.drop(columns = ['상품분류명'], axis=1)

item_categories

,상품분류id,대분류
0,0,0
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
79,79,0
80,80,0
81,81,0
82,82,0


In [9]:
# 데이터 조합 생성
from itertools import product

train = []

for i in sales_train['월id'].unique():
#     월id 에 해당하는 모든 상점 id를 선택함
    all_shop = sales_train.loc[sales_train['월id']==i , '상점id'].unique()
#     월 id에 해당하는 모든 상품 id를 선택함
    all_item = sales_train.loc[sales_train['월id']==i , '상품id'].unique()
    train.append(
        np.array(
#         produt를 사용하여 월id , 상점 id, 상품 id의 조합을 생성함
            list(product([i], all_shop , all_item))
         )
    )

#     기준피처설정
idx_features=['월id','상점id','상품id']

# list 타입의 train을 DataFrame 형태로 변경
train = pd.DataFrame(
#     각각의 list의 배열 요소를 세로로 붙임
    np.vstack(train),columns= idx_features
)

train
    

,월id,상점id,상품id
0,0,59,22154
1,0,59,2552
2,0,59,2554
3,0,59,2555
4,0,59,2564
...,...,...,...
10812763,33,21,7635
10812764,33,21,7638
10812765,33,21,7640
10812766,33,21,7632


In [10]:
## 파생 피처 생성
# 월간 판매량 / 평균 팡매가 피처 생성함

#     기준피처설정
idx_features=['월id','상점id','상품id']

group = sales_train.groupby(idx_features , as_index=False).agg(
#     월간판매량 생성
    월간판매량 = ('판매량','sum'),
#     평균판매가 생성
    평균판매가 = ('판매가','mean')
)

# train 데이터에 merge
train = train.merge(group , on=idx_features , how='left')

train.head()

,월id,상점id,상품id,월간판매량,평균판매가
0,0,59,22154,1.0,999.0
1,0,59,2552,NaN,NaN
2,0,59,2554,NaN,NaN
3,0,59,2555,NaN,NaN
4,0,59,2564,NaN,NaN


In [11]:
import gc
del group
gc.collect()

100

In [12]:
## 기준 피처별 상품 판매건수
#     기준피처설정
idx_features=['월id','상점id','상품id']

group = sales_train.groupby(idx_features , as_index=False).agg(
    판매건수 = ('판매량','count')
)

# train과 병합
train = train.merge(group , on=idx_features , how='left')

train.head()

,월id,상점id,상품id,월간판매량,평균판매가,판매건수
0,0,59,22154,1.0,999.0,1.0
1,0,59,2552,NaN,NaN,NaN
2,0,59,2554,NaN,NaN,NaN
3,0,59,2555,NaN,NaN,NaN
4,0,59,2564,NaN,NaN,NaN


In [13]:
## test data 이어 붙이기
# test data에 없는 월 id 피처 생성
test['월id']=34

# train과 test 이어 붙이기
all_data = pd.concat(
    [train, test.drop(columns=['ID'],axis=1)],
    ignore_index=True,
#     이어 붙이는 데 기준이 되는 피처
    keys=idx_features
)

# 앞에서 생긴 결측치를 0으로 대체
all_data=all_data.fillna(0)
all_data

,월id,상점id,상품id,월간판매량,평균판매가,판매건수
0,0,59,22154,1.0,999.0,1.0
1,0,59,2552,0.0,0.0,0.0
2,0,59,2554,0.0,0.0,0.0
3,0,59,2555,0.0,0.0,0.0
4,0,59,2564,0.0,0.0,0.0
...,...,...,...,...,...,...
11026963,34,45,18454,0.0,0.0,0.0
11026964,34,45,16188,0.0,0.0,0.0
11026965,34,45,15757,0.0,0.0,0.0
11026966,34,45,19648,0.0,0.0,0.0


In [14]:
### 데이터 병합하기
all_data = all_data.merge(shops , on='상점id', how='left')
all_data = all_data.merge(items , on='상품id' , how='left')
all_data = all_data.merge(item_categories , on='상품분류id' , how='left')


#데이터 다운캐스팅
all_data=downcast(all_data)

# 가비지 컬렉션
# import gc
# del shops , items , item_categories
# gc.collect

all_data.head()


all_data_temp=all_data.copy()
all_data_temp

 57.5% 압축됨


,월id,상점id,상품id,월간판매량,평균판매가,판매건수,도시,상품명,상품분류id,첫판매월,대분류
0,0,59,22154,1,999.0,1,30,ЯВЛЕНИЕ 2012 (BD),37,34,5
1,0,59,2552,0,0.0,0,30,DEEP PURPLE The House Of Blue Light LP,58,0,7
2,0,59,2554,0,0.0,0,30,DEEP PURPLE Who Do You Think We Are LP,58,2,7
3,0,59,2555,0,0.0,0,30,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,4,7
4,0,59,2564,0,0.0,0,30,DEEP PURPLE Perihelion: Live In Concert DVD (К...,59,2,7
...,...,...,...,...,...,...,...,...,...,...,...
11026963,34,45,18454,0,0.0,0,20,СБ. Союз 55,55,27,7
11026964,34,45,16188,0,0.0,0,20,Настольная игра Нано Кёрлинг,64,0,8
11026965,34,45,15757,0,0.0,0,20,НОВИКОВ АЛЕКСАНДР Новая коллекция,55,0,7
11026966,34,45,19648,0,0.0,0,20,ТЕРЕМ - ТЕРЕМОК сб.м/ф (Регион),40,33,5


In [15]:
## 시차 피처 생성
def add_mean_features(df, mean_features, idx_features):
    # 기준피처확인
    # assert 뒤에나오는 값이 false면 에러생김
    assert ((idx_features[0]=='월id') and len(idx_features) in [2,3])
    
    # 파생 피처명 설정
    if len(idx_features)==2:
        feature_name = idx_features[1] + '별 평균 판매량'
    else:
        feature_name = idx_features[1]+""+idx_features[2]+'별 평균 판매량'
    
    #기준 피처를 토대로 그륩화해 월간 평간 평균 판매량 구하기
    group = df.groupby(idx_features , as_index=False).agg(
       {'월간판매량' : 'mean'}
    )
    group=group.rename(columns ={
        '월간판매량' : feature_name
    })
    
    #df 와 group 병합
    df=df.merge(group , on=idx_features , how='left')
    
    #데이터 다운 캐스팅
    df= downcast(df, verbose=False)

    #새로 만든 feature_name 피처명을 mean_features 리스트에 추가
    mean_features.append(feature_name)
    
#     #가비지 컬렉션
#     del group 
#     gc.collect()
    
    return df, mean_features

In [16]:
# 그륩화 기준 피처 중 상품 id가 포함된 파생 피처명을 담을 리스트
item_mean_features=[]

# ['월 id' , '상품id'] 로 그륩화한 월간 평균 판매량 파생 피처 생성
all_data, item_mean_features = add_mean_features(
    df= all_data,
    mean_features=item_mean_features,
    idx_features=['월id', '상품id']
)

# ['월 id' , '상품id' ,' 도시'] 로 그륩화한 월간 평균 판매량 파생 피처 생성
all_data, item_mean_features = add_mean_features(
    df= all_data,
    mean_features=item_mean_features,
    idx_features=['월id', '상품id','도시']
)

all_data

,월id,상점id,상품id,월간판매량,평균판매가,판매건수,도시,상품명,상품분류id,첫판매월,대분류,상품id별 평균 판매량,상품id도시별 평균 판매량
0,0,59,22154,1,999.0,1,30,ЯВЛЕНИЕ 2012 (BD),37,34,5,0.400000,1.0
1,0,59,2552,0,0.0,0,30,DEEP PURPLE The House Of Blue Light LP,58,0,7,0.022222,0.0
2,0,59,2554,0,0.0,0,30,DEEP PURPLE Who Do You Think We Are LP,58,2,7,0.022222,0.0
3,0,59,2555,0,0.0,0,30,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,4,7,0.044444,0.0
4,0,59,2564,0,0.0,0,30,DEEP PURPLE Perihelion: Live In Concert DVD (К...,59,2,7,0.111111,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11026963,34,45,18454,0,0.0,0,20,СБ. Союз 55,55,27,7,0.000000,0.0
11026964,34,45,16188,0,0.0,0,20,Настольная игра Нано Кёрлинг,64,0,8,0.000000,0.0
11026965,34,45,15757,0,0.0,0,20,НОВИКОВ АЛЕКСАНДР Новая коллекция,55,0,7,0.000000,0.0
11026966,34,45,19648,0,0.0,0,20,ТЕРЕМ - ТЕРЕМОК сб.м/ф (Регион),40,33,5,0.000000,0.0


In [17]:
# 그륩화 기준 피처 중 상품 id가 포함된 파생 피처명을 담을 리스트
shop_mean_features=[]


# ['월 id' , '상품id' ,' 상품분류id'] 로 그륩화한 월간 평균 판매량 파생 피처 생성
all_data, shop_mean_features = add_mean_features(
    df= all_data,
    mean_features=shop_mean_features,
    idx_features=['월id', '상점id','상품분류id']
)

all_data

,월id,상점id,상품id,월간판매량,평균판매가,판매건수,도시,상품명,상품분류id,첫판매월,대분류,상품id별 평균 판매량,상품id도시별 평균 판매량,상점id상품분류id별 평균 판매량
0,0,59,22154,1,999.0,1,30,ЯВЛЕНИЕ 2012 (BD),37,34,5,0.400000,1.0,0.088496
1,0,59,2552,0,0.0,0,30,DEEP PURPLE The House Of Blue Light LP,58,0,7,0.022222,0.0,0.000000
2,0,59,2554,0,0.0,0,30,DEEP PURPLE Who Do You Think We Are LP,58,2,7,0.022222,0.0,0.000000
3,0,59,2555,0,0.0,0,30,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,4,7,0.044444,0.0,0.008333
4,0,59,2564,0,0.0,0,30,DEEP PURPLE Perihelion: Live In Concert DVD (К...,59,2,7,0.111111,0.0,0.011976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11026963,34,45,18454,0,0.0,0,20,СБ. Союз 55,55,27,7,0.000000,0.0,0.000000
11026964,34,45,16188,0,0.0,0,20,Настольная игра Нано Кёрлинг,64,0,8,0.000000,0.0,0.000000
11026965,34,45,15757,0,0.0,0,20,НОВИКОВ АЛЕКСАНДР Новая коллекция,55,0,7,0.000000,0.0,0.000000
11026966,34,45,19648,0,0.0,0,20,ТЕРЕМ - ТЕРЕМОК сб.м/ф (Регион),40,33,5,0.000000,0.0,0.000000


In [18]:
all_data_orgin=all_data.copy()

In [19]:
## 시차 피처 생성 원리 및 함수 구현

def add_lag_features(df, lag_features_to_clip , idx_features , lag_feature, nlags=3 , clip=False):
    # 시차 피처 생성에 필요한 DataFrame 부분복사
    df_temp = df[idx_features + [lag_feature]].copy()
    
    # 시차 피처 생성
    for i in range(1,nlags+1):
        # 시차 피처명
        lag_feature_name = f'{lag_feature}시차_{i}'
        #df_temp 열 이름 설정
        df_temp.columns=idx_features + [lag_feature_name]
        #df_temp의 date_block_num 피처에 i 더하기
        df_temp['월id']+=i
        #idx feature를 기준으로 df와 df_temp 병합하기
        df=df.merge(df_temp.drop_duplicates(),
                   on=idx_features,
                   how='left'
        )
        # 결측값 0으로 대체
        df[lag_feature_name] = df[lag_feature_name].fillna(0)
        
#         0~20 사이로 제한할 피처명을 lag_feature_to_clip에 추가
        if clip:
            lag_features_to_clip.append(lag_feature_name)
#     데이터 다운캐스트
    df=downcast(df,False)
    #가비지 컬렉션
    del df_temp
    gc.collect()
    
    return df , lag_features_to_clip

In [20]:
## 시차 생성 - 월간판매량
lag_features_to_clip=[]
# 기준피처
idx_features=['월id','상점id','상품id']

# idx_features를 기준으로 월간 판매량의 세달치 시차 피처 생성
all_data , lag_features_to_clip =add_lag_features(
            df=all_data,
            lag_features_to_clip = lag_features_to_clip,
            idx_features = idx_features,
            lag_feature='월간판매량',
            nlags=3,
#     월간판매량은 타깃값이므로 0~20 제한 필요함 --> clip=True로 하여 lag_features_to_clip에 변수저장함
            clip=True

)

In [21]:
## 시차생성 - 판매건수 , 평균 판매가
# 기준피처
idx_features=['월id','상점id','상품id']

# idx_features를 기준으로 월간 판매건수의 세달치 시차 피처 생성
all_data , lag_features_to_clip =add_lag_features(
            df=all_data,
            lag_features_to_clip = lag_features_to_clip,
            idx_features = idx_features,
            lag_feature='판매건수',
            nlags=3,
#     판매건수는 타겟값이 아니므로 값을 고정할 필요없음
            clip=False
)

# idx_features를 기준으로 월간 판매량의 세달치 시차 피처 생성
all_data , lag_features_to_clip =add_lag_features(
            df=all_data,
            lag_features_to_clip = lag_features_to_clip,
            idx_features = idx_features,
            lag_feature='평균판매가',
            nlags=3,
#     평균판매가는 타겟값이 아니므로 값을 고정할 필요없음
            clip=False
)

all_data.head().T

,0,1,2,3,4
월id,0,0,0,0,0
상점id,59,59,59,59,59
상품id,22154,2552,2554,2555,2564
월간판매량,1,0,0,0,0
평균판매가,999.0,0.0,0.0,0.0,0.0
판매건수,1,0,0,0,0
도시,30,30,30,30,30
상품명,ЯВЛЕНИЕ 2012 (BD),DEEP PURPLE The House Of Blue Light LP,DEEP PURPLE Who Do You Think We Are LP,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),DEEP PURPLE Perihelion: Live In Concert DVD (К...
상품분류id,37,58,58,56,59
첫판매월,34,0,2,4,2


In [22]:
## 시차생성 - 평균 판매가
# 기준피처
idx_features=['월id','상점id','상품id']

##앞에서 저장한   
# 1) 상품 id별 평균 판매량 => group =['월id' , '상품id']                               ------  item_mean_features

# 2) 상품 id별 도시별 평균 판매량 => group =['월id' , '상품id' ,'도시']                 ------  item_mean_features
  
# 3) 상품id 별 상품분류별 평균 판매량 => group =['월id' ,'상점id' , '상품분류id']        ------   shop_mean_features
# 에 대한 시차적용함

for item_mean_featrue in item_mean_features:
    # idx_features를 기준으로 월간 판매건수의 세달치 시차 피처 생성
    all_data , lag_features_to_clip =add_lag_features(
                df=all_data,
                lag_features_to_clip = lag_features_to_clip,
                idx_features = idx_features,
                lag_feature=item_mean_featrue,
                nlags=3,
    #     위에 id별 평균 판매량과 , 도시별 평균판매량은 타깃값과 같이 월평균판매량이므로 0~20 제한 필요함
                clip=False
    )
    
# 시차를 위해 사용하고 기존에 있는 1) , 2)의 변수제거
all_data = all_data.drop(columns=item_mean_features , axis=1)
all_data.head().T

,0,1,2,3,4
월id,0,0,0,0,0
상점id,59,59,59,59,59
상품id,22154,2552,2554,2555,2564
월간판매량,1,0,0,0,0
평균판매가,999.0,0.0,0.0,0.0,0.0
판매건수,1,0,0,0,0
도시,30,30,30,30,30
상품명,ЯВЛЕНИЕ 2012 (BD),DEEP PURPLE The House Of Blue Light LP,DEEP PURPLE Who Do You Think We Are LP,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),DEEP PURPLE Perihelion: Live In Concert DVD (К...
상품분류id,37,58,58,56,59
첫판매월,34,0,2,4,2


In [23]:
for shop_mean_featrue in shop_mean_features:
    # idx_features를 기준으로 월간 판매건수의 세달치 시차 피처 생성
    all_data , lag_features_to_clip =add_lag_features(
                df=all_data,
                lag_features_to_clip = lag_features_to_clip,
                idx_features = idx_features,
                lag_feature=shop_mean_featrue,
                nlags=3,
    #     3)에 대항하는 값으로 타겟값에 영향을 주는 값
                clip=True
    )
    
# 시차를 위해 사용하고 기존에 있는 1) , 2)의 변수제거
all_data = all_data.drop(columns=shop_mean_features , axis=1)
all_data.head().T

,0,1,2,3,4
월id,0,0,0,0,0
상점id,59,59,59,59,59
상품id,22154,2552,2554,2555,2564
월간판매량,1,0,0,0,0
평균판매가,999.0,0.0,0.0,0.0,0.0
판매건수,1,0,0,0,0
도시,30,30,30,30,30
상품명,ЯВЛЕНИЕ 2012 (BD),DEEP PURPLE The House Of Blue Light LP,DEEP PURPLE Who Do You Think We Are LP,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),DEEP PURPLE Perihelion: Live In Concert DVD (К...
상품분류id,37,58,58,56,59
첫판매월,34,0,2,4,2


In [24]:
all_data_origin=all_data.copy()

In [25]:
all_data.loc[all_data['월id']<3].index

Int64Index([      0,       1,       2,       3,       4,       5,       6,
                  7,       8,       9,
            ...
            1122376, 1122377, 1122378, 1122379, 1122380, 1122381, 1122382,
            1122383, 1122384, 1122385],
           dtype='int64', length=1122386)

In [26]:
all_data=all_data.drop(all_data.loc[all_data['월id']<3].index)

In [27]:
all_data['월id'].unique()

array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34],
      dtype=int8)

In [28]:
## 월간 판매량 시차 피처들의 평균

all_data['월간판매량시차평균']=all_data[['월간판매량시차_1','월간판매량시차_2','월간판매량시차_3']].mean(axis=1)

#0~20 사이로 값 제한
all_data[lag_features_to_clip + ['월간판매량','월간판매량시차평균']] =\
all_data[lag_features_to_clip + ['월간판매량','월간판매량시차평균']].clip(0,20)

In [29]:
#시차 변화량
all_data['시차변화량1']=all_data['월간판매량시차_1'] / all_data['월간판매량시차_2']
all_data['시차변화량1'] = all_data['시차변화량1'].replace([np.inf , -np.inf], np.nan).fillna(0)

all_data['시차변화량2']=all_data['월간판매량시차_2'] / all_data['월간판매량시차_3']
all_data['시차변화량2'] = all_data['시차변화량2'].replace([np.inf , -np.inf], np.nan).fillna(0)

In [30]:
## 신상여부
all_data['신상여부'] = all_data['첫판매월']==all_data['월id']

## 첫판매후 경과기간
all_data['첫판매후기간']=all_data['월id'] - all_data['첫판매월']

#월
all_data['월'] = all_data['월id'] % 12

In [31]:
## 필요없는 피처 제거
all_data=all_data.drop(['첫판매월','평균판매가','판매건수','상품명'], axis=1)
all_data=downcast(all_data,False)

In [32]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9904582 entries, 1122386 to 11026967
Data columns (total 31 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   월id                     int8   
 1   상점id                    int8   
 2   상품id                    int16  
 3   월간판매량                   int8   
 4   도시                      int8   
 5   상품분류id                  int8   
 6   대분류                     int8   
 7   월간판매량시차_1               int8   
 8   월간판매량시차_2               int8   
 9   월간판매량시차_3               int8   
 10  판매건수시차_1                int8   
 11  판매건수시차_2                int8   
 12  판매건수시차_3                int8   
 13  평균판매가시차_1               float32
 14  평균판매가시차_2               float32
 15  평균판매가시차_3               float32
 16  상품id별 평균 판매량시차_1        float32
 17  상품id별 평균 판매량시차_2        float32
 18  상품id별 평균 판매량시차_3        float32
 19  상품id도시별 평균 판매량시차_1      float32
 20  상품id도시별 평균 판매량시차_2      float32
 21  상품id도시별 평균 판매량시차_3      

In [33]:
all_data['월id'].unique()

array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34],
      dtype=int8)

In [34]:
## 데이터 분리
#훈련데이터
X_train = all_data.loc[all_data['월id'] < 33]
X_train = X_train.drop(columns = ['월간판매량'], axis=1)

#검증데이터
X_valid = all_data.loc[all_data['월id'] == 33]
X_valid = X_valid.drop(columns = ['월간판매량'], axis=1)

#테스트데이터
X_test = all_data.loc[all_data['월id'] == 34]
X_test = X_test.drop(columns = ['월간판매량'], axis=1)



#타켓값 설정
# 훈련데이터
y_train = all_data.loc[all_data['월id'] < 33 , '월간판매량']
# 판매량값을 0~20으로 제한시킴 <- 이번 경진대회 공지사항에 나와있음
y_train = y_train.clip(0,20)


# 검증데이터
y_valid = all_data.loc[all_data['월id'] == 33 , '월간판매량']
# 판매량값을 0~20으로 제한시킴 <- 이번 경진대회 공지사항에 나와있음
y_valid = y_valid.clip(0,20)


print("훈련데이터" , X_train.shape)
print("검증데이터" , X_valid.shape)
print("테스트" , X_test.shape)

print("타켓_훈련" , y_train.shape)
print("타켓 검증", y_valid.shape)

훈련데이터 (9452298, 30)
검증데이터 (238084, 30)
테스트 (214200, 30)
타켓_훈련 (9452298,)
타켓 검증 (238084,)


In [35]:
#light gbm 사용하기

import lightgbm as lgb

# 파라미터
params={
    'metric' : 'rmse',
    'num_leaves': 255,
    'learning_rate' : 0.005,
    'feature_fraction':0.75,
    'bagging_fraction':0.75,
    'bagging_freq':5,
    'force_col_wise' : True,
    'random_state' : 10
}

# 범주형 피처설정
cat_features = ['상점id' , '상품분류id' ,'도시' ,'대분류', '월']

# lightGbm 용 훈련 및 검증 데이터 셋
dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid , y_valid)

##모델훈련
lgb_model = lgb.train(
                        params = params,
                        train_set = dtrain,
                        num_boost_round=1500,
                        valid_sets=(dtrain,dvalid),
                        early_stopping_rounds=150,
                        categorical_feature = cat_features,
                        verbose_eval=100
                      )

Training until validation scores don't improve for 150 rounds.
[100]	training's rmse: 1.02277	valid_1's rmse: 1.00447
[200]	training's rmse: 0.926274	valid_1's rmse: 0.946791
[300]	training's rmse: 0.877723	valid_1's rmse: 0.925661
[400]	training's rmse: 0.850269	valid_1's rmse: 0.918397
[500]	training's rmse: 0.832258	valid_1's rmse: 0.916715
[600]	training's rmse: 0.819947	valid_1's rmse: 0.916684
Early stopping, best iteration is:
[519]	training's rmse: 0.8297	valid_1's rmse: 0.916467


In [36]:
## 예측 및 결과제출
#예측
preds = lgb_model.predict(X_test).clip(0,20)

#제출파일생성
submission['item_cnt_month']=preds
submission.to_csv('submission.csv',index=False)